<a href="https://colab.research.google.com/github/pedrojmlmelo/VeracIA/blob/main/Verificador_de_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Verificador de Fonte --- #
##########################################

def agente_verificador_fonte(url) :
  fonte = Agent(
      name = "agente_verificador_fonte",
      model = "gemini-2.0-flash",
      instruction = """
            Você é um especialista em análise de credibilidade de fontes de notícias na internet.
            Sua tarefa é avaliar a confiabilidade da fonte com base na URL fornecida.
            Considere fatores como: reputação do domínio, se é conhecido por espalhar desinformação,
            se possui uma linha editorial clara, transparência sobre autores e propriedade.
            Seja conciso e direto na sua avaliação.
      """,
      description = "Agente que verifica a fonte da noticia",
      tools = [google_search]
)

  entrada_agente_verificador_fonte = f"Url: {url}\n"
  # Executa o agente
  fonte_verificada = call_agent(fonte, entrada_agente_verificador_fonte)
  return fonte_verificada


In [ ]:
##########################################
# --- Agente 2: Analisador de Conteudo --- #
##########################################

def agente_analisador_conteudo(fonte_verificada) :
  conteudo = Agent(
      name = "agente_analisador_conteudo",
      model = "gemini-2.0-flash",
      instruction = """
            "Você é um especialista em identificar técnicas de desinformação e manipulação em textos. "
            "Analise o texto da notícia fornecida e aponte possíveis sinais de alerta, como: "
            "linguagem excessivamente emocional ou alarmista, generalizações apressadas, "
            "falta de fontes ou evidências concretas, apelo a teorias da conspiração, "
            "erros grosseiros de português ou formatação que sugiram falta de profissionalismo, "
            "ou alegações extraordinárias sem provas extraordinárias. Seja objetivo.
      """,
      description = "Agente que analisa o conteudo da noticia",
      tools = [google_search]
)

  entrada_agente_analisador_conteudo = fonte_verificada
  # Executa o agente
  conteudo_analisado = call_agent(conteudo, entrada_agente_analisador_conteudo)
  return conteudo_analisado

In [ ]:
##########################################
# --- Agente 3: Checagem cruzada --- #
##########################################

def agente_checagem_cruzada(conteudo_analisado) :
  cruzada = Agent(
      name = "agente_checagem_cruzada",
      model = "gemini-2.0-flash",
      instruction = """
            "Você é um assistente de checagem de fatos (fact-checker) muito rigoroso. "
            "Sua tarefa é verificar a veracidade das principais alegações contidas no texto da notícia. "
            "Use seu conhecimento para buscar informações que corroborem ou contradigam essas alegações. "
            "Mencione se há consenso sobre o tema, se é uma informação disputada ou se é amplamente considerada falsa. "
            "Se a notícia for sobre um evento muito recente ou obscuro, admita se não encontrar informações. "
            "Indique fontes confiáveis se possível (conceitualmente, você não navega na web em tempo real, mas use seu conhecimento)
      """,
      description = "Agente que faz a analise cruzada",
      tools = [google_search]
)

  entrada_agente_checagem_cruzada = conteudo_analisado
  # Executa o agente
  cruzada_analisado = call_agent(cruzada, entrada_agente_checagem_cruzada)
  return cruzada_analisado

In [ ]:
##########################################
# --- Agente 4: Sintetizador --- #
##########################################

def agente_sintetizador(cruzada_analisado) :
  sintetizador = Agent(
      name = "agente_sintetizador",
      model = "gemini-2.0-flash",
      instruction = """
            "Você é um assistente de IA sênior encarregado de consolidar as análises de outros agentes de IA "
            "para fornecer um parecer final sobre a probabilidade de uma notícia ser fake news. "
            "Considere todos os relatórios: coleta de dados, análise da fonte, análise do conteúdo e checagem cruzada. "
            "Seu parecer deve ser claro, indicar um nível de confiança (ex: 'Alta probabilidade de ser fake news', "
            "'Potencialmente enganosa', 'Informação parece precisa', 'Inconclusivo') e "
            "justificar brevemente com base nos pontos mais críticos dos relatórios."
            "Seja cauteloso e equilibrado em seu julgamento."
            "Sua resposta deve conter no final uma nota que classifique entre:
            'Alta probabilidade de ser fake news',
            'Potencialmente enganosa',
            'Informação parece precisa',
            'Inconclusivo'
      """,
      description = "Agente que faz a sintetização",
      tools = [google_search]
)

  entrada_agente_sintetizador = cruzada_analisado
  # Executa o agente
  sintetizada_analisado = call_agent(sintetizador, entrada_agente_sintetizador)
  return sintetizada_analisado

In [ ]:
# --- Obter a URL do Usuário ---
url = input("Por favor, digite a url da noticia: ")


fonte_check = agente_verificador_fonte(url)
# print("\n--- resultados agente 1 ---")
# display(to_markdown(fonte_check))
# print("--------------------")

conteudo_check = agente_analisador_conteudo(fonte_check)
# print("\n--- resultados agente 2 ---")
# display(to_markdown(conteudo_check))
# print("--------------------")

cruzada_check = agente_checagem_cruzada(conteudo_check)
# print("\n--- resultados agente 3 ---")
# display(to_markdown(cruzada_check))
# print("--------------------")

sintetizador_check = agente_sintetizador(cruzada_check)
print("\n--- resultados agente 4 ---")
display(to_markdown(sintetizador_check))
print("--------------------")